!pip install rdkit-pypi

In [2]:
# import statements
from rdkit import Chem
from rdkit.Chem import DataStructs, AllChem

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # Functipn to split data into training, validation and test sets
import pickle
import glob   # The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order. No tilde expansion is done, but *, ?, and character ranges expressed with [] will be correctly matched.
import os   # miscellneous operating system interfaces. This module provides a portable way of using operating system dependent functionality. If you just want to read or write a file see open(), if you want to manipulate paths, see the os.path module, and if you want to read all the lines in all the files on the command line see the fileinput module.
import random       
from tqdm import tqdm 
from tqdm.notebook import tqdm_notebook
import datetime

# Torch
import torch
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn


RDKit WARNING: [09:45:35] Enabling RDKit 2019.09.1 jupyter extensions


In [3]:
now = datetime.datetime.now()
now = now.strftime("%d_%m_%Y-%H:%M:%S")

# Pseudo Code
1. download train, validation and test data
2. keep only columns with relevant data (compound name, smile strings, moa)
    1. assert that dmso not in all data values
3. create function that gets produces a dictionary with all relevant moas and assigns a number to them
    1. extract unique values
    2. enumerate loop, add to growing dictionary, where name is the key.
4. set these values as moa
5. do one hot encoding
6. Fix torch_set so that it can handle pandas instead of an extra dictionary 

In [4]:
compounds_v1v2 = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/compounds_v1v2.csv', delimiter = ",")
    

In [5]:
# testing using pandas dataframe
training_set = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/CS_data_splits/CS_training_set_cyclo_adr_2.csv')
validation_set = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/CS_data_splits/CS_valid_set_cyclo_adr_2.csv')
test_set = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/CS_data_splits/CS_test_set_cyclo_adr_2.csv')

In [14]:
test_set.moa.value_counts()

0    10
1     4
Name: moa, dtype: int64

In [10]:
with open('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/CS_data_splits/cyclo_adr_2_moa_dict.pickle', 'rb') as handle:
        moa_dict = pickle.load(handle)
moa_dict

{'cyclooxygenase inhibitor': 0, 'dopamine receptor antagonist': 1}

In [12]:
target= [None]*len(moa_dict)
for i in moa_dict.items():
    target[i[1]] = i[0]

In [13]:
target

['cyclooxygenase inhibitor', 'dopamine receptor antagonist']

In [6]:
def splitting_into_tensor(df, num_classes):
    '''Splitting data into two parts:
    1. input : the pointer showing where the transcriptomic profile is  
    2. target one hot : labels (the correct MoA) '''
    
    # one-hot encoding labels
     # creating tensor from all_data.df
    target = torch.tensor(df['moa'].values.astype(np.int64))

    # For each row, take the index of the target label
    # (which coincides with the score in our case) and use it as the column index to set the value 1.0.” 
    #target_onehot = torch.zeros(target.shape[0], num_classes)
    #target_onehot.scatter_(1, target.unsqueeze(1), 1.0)
    
    input =  df.drop('moa', axis = 1)
    
    return input, target #target_onehot

In [7]:
assert training_set.moa.unique().all() == validation_set.moa.unique().all() == test_set.moa.unique().all()

In [8]:
training_set.moa.unique()

array([0, 1])

In [474]:
num_classes = len(training_set.moa.unique())
num_classes

2

In [475]:
training_df, train_labels = splitting_into_tensor(training_set, num_classes)
validation_df, validation_labels = splitting_into_tensor(validation_set, num_classes)
test_df, test_labels = splitting_into_tensor(test_set, num_classes)

In [476]:
# got to put the labels into the same array as compounds_moa in order for the indexing to work. Biggest problem is I only have
# have 250 compounds to train, not 120000, which probabilty artificially gives a bunch of extra compounds I don't have

In [477]:
batch_size = 50
# parameters
params = {'batch_size' : batch_size,
         'num_workers' : 3,
         'shuffle' : True,
         'prefetch_factor' : 1} 
          
# shuffle isn't working

# Datasets
#partition = partition
#labels = labels

# maxepochs
max_epochs = 250

In [478]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
#device = torch.device('cpu')
print(f'Training on device {device}. ' )

Training on device cpu. 


In [479]:
# A function changing SMILES to Morgan fingerprints 
def smiles_to_array(smiles):
    molecules = Chem.MolFromSmiles(smiles) 
    fingerprints = AllChem.GetMorganFingerprintAsBitVect(molecules, 2)
    x_array = []
    arrays = np.zeros(0,)
    DataStructs.ConvertToNumpyArray(fingerprints, arrays)
    x_array.append(arrays)
    x_array = np.asarray(x_array)
    x_array = ((np.squeeze(x_array)).astype(int)) 
    x_array = torch.from_numpy(x_array)
    return x_array                  

In [480]:
# create Torch.dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, compound_df, labels, transform=None):
        self.compound_labels = labels    # the entire length of the correct classes that we are trying to predict
        # print(self.img_labels)
        self.compound_df = compound_df        # list of indexes that are a part of training, validation, tes sets
        self.transform = transform       # any transformations done

    def __len__(self):
        ''' The number of data points'''
        return len(self.compound_labels)      

    def __getitem__(self, idx):
        '''Retrieving the compound '''
        #print(idx)
        smile_string = self.compound_df["SMILES"][idx]      # returns smiles by using compound as keys
        #print(smile_string)
        compound_array = smiles_to_array(smile_string)
        #print(f' return from function: {compound}')
        #print(f' matrix: {compound_array}')
        label = self.compound_labels[idx]             # extract classification using index
        #print(f' label: {label}')
        #label = torch.tensor(label, dtype=torch.float)
        if self.transform:                         # uses Albumentations image pipeline to return an augmented image
            compound = self.transform(compound)
        return compound_array.float(), label.long()

## Generators

In [481]:
# Create a dataset with all indices and labels
training_dataset = Dataset(training_df, train_labels)
valid_dataset = Dataset(validation_df, validation_labels)
test_dataset = Dataset(test_df, test_labels)

In [482]:
assert len(training_df) == len(train_labels)
assert len(validation_df) == len(validation_labels)
assert len(test_df) == len(test_labels)

In [483]:
# create generator that randomly takes indices from the training set
training_generator = torch.utils.data.DataLoader(training_dataset, **params)
validation_generator = torch.utils.data.DataLoader(valid_dataset, **params)
test_generator = torch.utils.data.DataLoader(test_dataset, **params)
# training_set = Dataset(partition["train"], labels)

In [484]:
#partition["train"]

In [485]:
# training data loader
# Display image and label   # functional
train_features, train_labels = next(iter(training_generator))
print(f"Feature batch shape: {train_features}")
print(f"Labels batch shape: {train_labels}")

Feature batch shape: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Labels batch shape: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0])


In [486]:
valid_features, valid_labels = next(iter(validation_generator))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([50, 2048])
Labels batch shape: torch.Size([50])


In [487]:
test_features, test_labels = next(iter(test_generator))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([50, 2048])
Labels batch shape: torch.Size([50])


In [488]:
# If applying class weights
apply_class_weights = True
if apply_class_weights:     # if we want to apply class weights
    counts = training_set.moa.value_counts()  # count the number of moa in each class for the ENTiRE dataset
    print(counts)
    class_weights = []   # create list that will hold class weights
    for moa in training_set.moa.unique():       # for each moa   
        #print(moa)
        counts[moa]
        class_weights.append(counts[moa])  # add counts to class weights
    print(len(class_weights))
    print(class_weights)
    print(type(class_weights))
    # class_weights = 1 / (class_weights / sum(class_weights)) # divide all class weights by total moas
    class_weights = [i / sum(class_weights) for  i in class_weights]
    class_weights= torch.tensor(class_weights,dtype=torch.float).to(device) # transform into tensor, put onto device
print(class_weights)

0    45
1    18
Name: moa, dtype: int64
2
[45, 18]
<class 'list'>
tensor([0.7143, 0.2857])


In [489]:
# Creating Architecture
units = 64
drop  = 0.7

seq_model = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(),
    nn.Dropout(p = drop),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64, num_classes))

In [490]:
seq_model

Sequential(
  (0): Linear(in_features=2048, out_features=128, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.7, inplace=False)
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Linear(in_features=64, out_features=2, bias=True)
)

In [491]:
# optimizer_algorithm
#cnn_optimizer = torch.optim.Adam(updated_model.parameters(),weight_decay = 1e-6, lr = 0.001, betas = (0.9, 0.999), eps = 1e-07)
optimizer = torch.optim.Adam(seq_model.parameters(), lr = 1e-4)
# loss_function
if apply_class_weights == True:
    loss_function = torch.nn.CrossEntropyLoss(class_weights)
else:
    loss_function = torch.nn.CrossEntropyLoss()

In [492]:
'''# complete the architecture of MLP and compile MLP 

units = 64  
drop = 0.89  

model_mlp = Sequential()
model_mlp.add(Dense(units, input_dim = 2048, activation = 'relu'))
model_mlp.add(Dropout(drop))
model_mlp.add(Dense(10, activation = 'softmax'))
model_mlp.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4),
         loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         metrics = ['accuracy'])'''

"# complete the architecture of MLP and compile MLP \n\nunits = 64  \ndrop = 0.89  \n\nmodel_mlp = Sequential()\nmodel_mlp.add(Dense(units, input_dim = 2048, activation = 'relu'))\nmodel_mlp.add(Dropout(drop))\nmodel_mlp.add(Dense(10, activation = 'softmax'))\nmodel_mlp.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4),\n         loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),\n         metrics = ['accuracy'])"

In [493]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, valid_loader):
    '''
    n_epochs: number of epochs 
    optimizer: optimizer used to do backpropagation
    model: deep learning architecture
    loss_fn: loss function
    train_loader: generator creating batches of training data
    valid_loader: generator creating batches of validation data
    '''
    # lists keep track of loss and accuracy for training and validation set
    #optimizer = torch.optim.Adam(updated_model.parameters(),weight_decay = 1e-6, lr = 0.001, betas = (0.9, 0.999), eps = 1e-07)
    train_loss_per_epoch = []
    train_acc_per_epoch = []
    val_loss_per_epoch = []
    val_acc_per_epoch = []
    best_val_loss = np.inf
    for epoch in tqdm(range(1, max_epochs +1), desc = "Epoch", position = 0, leave = True):
        loss_train = 0.0
        train_total = 0
        train_correct = 0
        for compounds, labels in train_loader:
            optimizer.zero_grad()
            # put model, images, labels on the same device
            model = model.to(device)
            compounds = compounds.to(device = device)
            labels = labels.to(device= device)
            #print(f' Compounds {compounds}')
            #print(f' Labels {labels}')
            
            #print(labels)
            # Training Model
            outputs = model(compounds)
            #print(f' Outputs : {outputs}') # tensor with 10 elements
            #print(f' Labels : {labels}') # tensor that is a number
            loss = loss_fn(outputs,labels)
            # Update weights
            loss.backward()
            optimizer.step()
            # Training Metrics
            loss_train += loss.item()
            #print(f' loss: {loss.item()}')
            train_predicted = torch.argmax(outputs, 1)
            #print(f' train_predicted {train_predicted}')
            # NEW
            #labels = torch.argmax(labels,1)
            #print(labels)
            train_total += labels.shape[0]
            train_correct += int((train_predicted == labels).sum())
        # validation metrics from batch
        val_correct, val_total, val_loss, best_val_loss_upd = validation_loop(model, loss_fn, valid_loader, best_val_loss, epoch)
        val_accuracy = val_correct/val_total
        # printing results for epoch
        if epoch == 1 or epoch %2 == 0:
            print(f' {datetime.datetime.now()} Epoch: {epoch}, Training loss: {loss_train/len(train_loader)}, Validation Loss: {val_loss} ')
        # adding epoch loss, accuracy to lists 
        val_loss_per_epoch.append(val_loss)
        train_loss_per_epoch.append(loss_train/len(train_loader))
        val_acc_per_epoch.append(val_accuracy)
        train_acc_per_epoch.append(train_correct/train_total)
    # return lists with loss, accuracy every epoch
    return train_loss_per_epoch, train_acc_per_epoch, val_loss_per_epoch, val_acc_per_epoch

In [494]:
def validation_loop(model, loss_fn, valid_loader, best_val_loss, epoch):
    '''
    Assessing trained model on valiidation dataset 
    model: deep learning architecture getting updated by model
    loss_fn: loss function
    valid_loader: generator creating batches of validation data
    '''
    loss_val = 0.0
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():  # does not keep track of gradients so as to not train on validation data.
        for compounds, labels in valid_loader:
            # Move to device MAY NOT BE NECESSARY
            model = model.to(device)
            compounds = compounds.to(device = device)
            labels = labels.to(device= device)
            # Assessing outputs
            outputs = model(compounds)
            #print(f' Outputs : {outputs}') # tensor with 10 elements
            #print(f' Labels : {labels}') # tensor that is a number
            loss = loss_fn(outputs,labels)
            loss_val += loss.item()
            predicted = torch.argmax(outputs, 1)
            #labels = torch.argmax(labels,1)
            #print(predicted)
            #print(labels)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
        avg_val_loss = loss_val/len(valid_loader)  # average loss over batch
        if best_val_loss > loss_val:
            best_val_loss = loss_val
            torch.save(
                {'epoch': epoch,
                    'model_state_dict' : model.state_dict(),
                    'valid_loss' : loss_val
            },  '/home/jovyan/Tomics-CP-Chem-MoA/01_CStructure_Models/saved_models/pre_split/' + 'ChemStruc_least_loss_model'
            )
    model.train()
    return correct, total, avg_val_loss, best_val_loss

In [495]:
def test_loop(model, loss_fn, test_loader):
    '''
    Assessing trained model on test dataset 
    model: deep learning architecture getting updated by model
    loss_fn: loss function
    test_loader: generator creating batches of test data
    '''
    loss_test = 0.0
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():  # does not keep track of gradients so as to not train on test data.
        for compounds, labels in tqdm_notebook(test_loader):
            # Move to device MAY NOT BE NECESSARY
            model = model.to(device)
            compounds = compounds.to(device = device)
            labels = labels.to(device= device)
            # Assessing outputs
            outputs = model(compounds)
            # print(f' Outputs : {outputs}') # tensor with 10 elements
            # print(f' Labels : {labels}') # tensor that is a number
            loss = loss_fn(outputs,labels)
            loss_test += loss.item()
            predicted = torch.argmax(outputs, 1)
            #labels = torch.argmax(labels,1)
            #print(predicted)
            #print(labels)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
        avg_test_loss = loss_test/len(test_loader)  # average loss over batch
    return correct, total, avg_test_loss

In [496]:
train_loss_per_epoch, train_acc_per_epoch, val_loss_per_epoch, val_acc_per_epoch = training_loop(n_epochs = max_epochs,
              optimizer = optimizer,
              model = seq_model,
              loss_fn = loss_function,
              train_loader=training_generator, 
              valid_loader=validation_generator)

Epoch:   0%|          | 1/250 [00:00<01:51,  2.24it/s]

 2023-02-06 16:47:32.816562 Epoch: 1, Training loss: 0.7589018940925598, Validation Loss: 0.7469367980957031 


Epoch:   1%|          | 2/250 [00:00<02:02,  2.02it/s]

 2023-02-06 16:47:33.346247 Epoch: 2, Training loss: 0.7542061507701874, Validation Loss: 0.7446339726448059 


Epoch:   2%|▏         | 4/250 [00:01<02:01,  2.02it/s]

...

 2023-02-06 16:49:38.467138 Epoch: 246, Training loss: 0.011859591584652662, Validation Loss: 0.020989414304494858 


Epoch:  99%|█████████▉| 248/250 [02:07<00:01,  1.99it/s]

 2023-02-06 16:49:39.441997 Epoch: 248, Training loss: 0.006795366061851382, Validation Loss: 0.020632246509194374 


Epoch: 100%|██████████| 250/250 [02:08<00:00,  1.95it/s]

 2023-02-06 16:49:40.508095 Epoch: 250, Training loss: 0.006349924253299832, Validation Loss: 0.02028319425880909 


In [497]:
#print(train_loss_per_epoch, train_acc_per_epoch, val_loss_per_epoch, val_acc_per_epoch)

In [498]:
def val_vs_train_loss(epochs, train_loss, val_loss):
    ''' 
    Plotting validation versus training loss over time
    epochs: number of epochs that the model ran (int. hyperparameter)
    train_loss: training loss per epoch (python list)
    val_loss: validation loss per epoch (python list)
    ''' 
    loss_path_to_save = '/home/jovyan/Tomics-CP-Chem-MoA/01_CStructure_Models/saved_images/pre_split'
    plt.figure()
    x_axis = list(range(1, epochs +1)) # create x axis with number of
    plt.plot(x_axis, train_loss, label = "train_loss")
    plt.plot(x_axis, val_loss, label = "val_loss")
    # Figure description
    plt.xlabel('# of Epochs')
    plt.ylabel('Loss')
    plt.title('Validation versus Training Loss: CP Image Model')
    plt.legend()
    # plot
    plt.savefig(loss_path_to_save + '/' + 'loss_train_val_' + now)


In [499]:
def val_vs_train_accuracy(epochs, train_acc, val_acc):
    '''
    Plotting validation versus training loss over time
    epochs: number of epochs that the model ran (int. hyperparameter)
    train_acc: accuracy loss per epoch (python list)
    val_acc: accuracy loss per epoch (python list)
    '''
    acc_path_to_save = '/home/jovyan/Tomics-CP-Chem-MoA/01_CStructure_Models/saved_images/pre_split'
    plt.figure()
    x_axis = list(range(1, epochs +1)) # create x axis with number of
    plt.plot(x_axis, train_acc, label = "train_acc")
    plt.plot(x_axis, val_acc, label = "val_acc")
    # Figure description
    plt.xlabel('# of Epochs')
    plt.ylabel('Accuracy')
    plt.title('Validation versus Training Accuracy: CP Image Model')
    plt.legend()
    # plot
    plt.savefig(acc_path_to_save + '/' + 'acc_train_val_' + now)

In [500]:
val_vs_train_loss(max_epochs,train_loss_per_epoch, val_loss_per_epoch)

In [501]:
val_vs_train_accuracy(max_epochs, train_acc_per_epoch, val_acc_per_epoch)

In [502]:
correct, total, avg_test_loss = test_loop(model = seq_model,
                                          loss_fn = loss_function, 
                                          test_loader = test_generator)

  0%|          | 0/1 [00:00<?, ?it/s]

In [503]:
correct/total
# create PyTorch architecture

0.9285714285714286